# All shortest paths

All shortest path algorithm implemented by QUBO in quantum annealer. Classical version of this is for example bellman-ford: https://en.wikipedia.org/wiki/Bellman%E2%80%93Ford_algorithm

Here directed graph $G=(V,E)$ has sets of vertices $V$ and edges $E \subseteq \{(x,y)|(x,y) \in V^2 and x \not = y\}$. For every edge there is weight $w_{xy}$. Task is to find path with minimum sum of weights for defined $(s,t) \in E^2$.

Solution is inspired by "Directed Edge-Based Approach" from Kraus & McCollum (2020) Solving the Network Shortest Path Problem on a Quantum Annealer: https://doi.org/10.1109/TQE.2020.3021921

Penalty $p=\sum w_{xy}$.

Following constraints are build to QUBO: 
1. There should be one vertice starting from $s$ ($-p$ for each), no vertice should end to $s$ ($p$ for each)
2. There should be one vertice ending to $t$ ($-p$ for each), no vertice should start from $t$ ($p$ for each)
3. Two edges should not start/end to the same vertice, for example $s$ or $t$ (if so $2p$) 
4. Two edges should form a chain (ok chain gives $p$, otherwise more than that)
5. Path having lower weights should be prioritised

Proper path would give energy level of $w-p$ (constraint 4 brings minimum $p$ and constraint 1 and 2 both minimum $-p$). From this set of samples we choose sample(s) having lowest energy level. With perfectly working solver all samples with energy level below zero are correct paths between $(s,t)$.

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector
import networkx as nx
import random

## Function to create QUBO

E is array of tuples (1st vertice, 2nd vertice, weight), p is penalty, s is starting vertice and t is terminating vertice.

In [2]:
def create_qubo(E,p,s,t):
    edges = len(E)
    Q = np.zeros((edges,edges))

    # Constraint 1
    for i in range(edges):
        if E[i][1]==s:
            Q[i][i] += p
        if E[i][0]==s:
            Q[i][i] -= p
        
    # Constraint 2
    for i in range(edges):
        if E[i][0]==t:
            Q[i][i] += p
        if E[i][1]==t:
            Q[i][i] -= p

    # Constraint 3
    for i in range(edges):
        for j in range(i+1,edges):
            if E[i][0]==E[j][0] or E[i][1]==E[j][1]:
                Q[i,j] = 2*p

    # Constraint 4
    for i in range(edges):
        Q[i,i] += p
        for j in range(i+1,edges):
            if E[i][1]==E[j][0] or E[i][0]==E[j][1]:
                Q[i,j] -= p

    # Constraint 5
    for i,e in enumerate(E):
        Q[i][i] += e[2]        

    return Q

## Helper functions for results

In [3]:
def path_from_sample(sample,s,t,E):
    w = 0
    i = s
    path = [i]
    while i!=t:
        found = False
        for e in E:
            if e[0]==i and sample[str(e[0]) + '-' + str(e[1])]==1:
                i = e[1]
                path.append(i)
                w += e[2]
                found = True
        if not found:
            print('Path broken')
            break
    return (str(s)+'-'+str(t),path,w)
        

def result_info(sampleset,s,t,E):
    energy= sampleset.first.energy
    res = []
    for sample in sampleset.filter(lambda s: s.energy==energy):
        st, path, w = path_from_sample(sample,s,t,E)
        if st not in res:
            res.append((st,path,w))
    return res

def make_G(E,vertices):
    G = nx.DiGraph()
    G.add_nodes_from([0, vertices-1])
    for e in E:
        G.add_edge(e[0], e[1], weight=e[2])
    return(G)

## Run with simple graph

### Define graph

Input graph is array of tuples (1st vertice, 2nd vertice, weight)

In [4]:
E1 = np.array([(0, 2, 1), (2, 1, 2), (1, 3, 3), (3, 2, 4), (0, 1, 5), (3, 4, 3), (2, 4, 8)])
vertices1 = 5
s1 = 0
t1 = 4

Above graph visualised:

![](graph4.png)

### Construct max_count and labels variables

In [5]:
max_count1 = 0
labels1 = {}
for i,e in enumerate(E1):
    max_count1 += e[2]
    labels1[i] = str(e[0]) + '-' + str(e[1])
print('Max count:',max_count1)

Max count: 26


### Create QUBO and BQM

In [6]:
ts = time.time()
Q1 = create_qubo(E1,max_count1,s1,t1)
qubo_time = (time.time()-ts)*1000
print('Time used for construction Q (ms): {:.3f}\n'.format(qubo_time))
print(labels1)
print(Q1)

ts = time.time()
bqm1 = dimod.BinaryQuadraticModel(Q1, 'BINARY')
bqm_time = (time.time()-ts)*1000
bqm1 = bqm1.relabel_variables(labels1, inplace=False)

Time used for construction Q (ms): 0.235

{0: '0-2', 1: '2-1', 2: '1-3', 3: '3-2', 4: '0-1', 5: '3-4', 6: '2-4'}
[[  1. -26.   0.  52.  52.   0. -26.]
 [  0.  28. -26. -26.  52.   0.  52.]
 [  0.   0.  29. -26. -26. -26.   0.]
 [  0.   0.   0.  30.   0.  52. -26.]
 [  0.   0.   0.   0.   5.   0.   0.]
 [  0.   0.   0.   0.   0.   3.  52.]
 [  0.   0.   0.   0.   0.   0.   8.]]


### Local deterministic classical solver

In [7]:
ts = time.time()
sampleset = dimod.ExactSolver().sample(bqm1)
det_time = (time.time()-ts)*1000
print('Time used (ms): {:.3f}\n'.format(det_time))
energy = sampleset.first.energy
print(sampleset.filter(lambda s: s.energy<0))

Time used (ms): 1.717

  0-1 0-2 1-3 2-1 2-4 3-2 3-4 energy num_oc.
1   0   1   1   1   0   0   1  -17.0       1
3   0   1   0   0   1   0   0  -17.0       1
0   1   0   1   0   0   0   1  -15.0       1
2   1   0   1   0   1   1   0   -6.0       1
['BINARY', 4 rows, 4 samples, 7 variables]


In [8]:
for r in result_info(sampleset,s1,t1,E1):
    print('Route '+r[0]+': '+str(r[1])+', weight '+str(r[2]))

Route 0-4: [0, 2, 1, 3, 4], weight 9
Route 0-4: [0, 2, 4], weight 9


### Local heuristic classical solver

In [9]:
num_reads = 400
ts = time.time()
sampleset2 = SimulatedAnnealingSampler().sample(bqm1, num_reads=num_reads).aggregate()
heur_time = (time.time()-ts)*1000
print('Time used (ms): {:.3f}\n'.format(heur_time))
print(sampleset2)

Time used (ms): 90.060

  0-1 0-2 1-3 2-1 2-4 3-2 3-4 energy num_oc.
0   0   1   0   0   1   0   0  -17.0     160
2   0   1   1   1   0   0   1  -17.0     149
1   1   0   1   0   0   0   1  -15.0      83
3   1   0   1   0   1   1   0   -6.0       8
['BINARY', 4 rows, 400 samples, 7 variables]


In [10]:
for r in result_info(sampleset2,s1,t1,E1):
    print('Route '+r[0]+': '+str(r[1])+', weight '+str(r[2]))

Route 0-4: [0, 2, 4], weight 9
Route 0-4: [0, 2, 1, 3, 4], weight 9


### Quantum solver

In [11]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [12]:
num_reads=1000
sampleset3 = EmbeddingComposite(machine).sample(bqm1, num_reads=num_reads)

In [13]:
qpu_time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))
print(sampleset3.filter(lambda s: s.energy<0).aggregate())

QPU time used (ms): 202.1
Physical qubits used: 8

  0-1 0-2 1-3 2-1 2-4 3-2 3-4 energy num_oc. chain_.
0   0   1   0   0   1   0   0  -17.0     275     0.0
1   0   1   1   1   0   0   1  -17.0     232     0.0
2   1   0   1   0   0   0   1  -15.0     239     0.0
3   1   0   1   0   1   1   0   -6.0      56     0.0
['BINARY', 4 rows, 802 samples, 7 variables]


In [14]:
for r in result_info(sampleset3.aggregate(),s1,t1,E1):
    print('Route '+r[0]+': '+str(r[1])+', weight '+str(r[2]))

Route 0-4: [0, 2, 4], weight 9
Route 0-4: [0, 2, 1, 3, 4], weight 9


In [15]:
#dwave.inspector.show(sampleset3)

### Hybrid solver

Hybrid solver brings only one solution, not good here

In [16]:
#sampleset4 = LeapHybridSampler().sample(bqm)

In [17]:
#hyb_time = sampleset4.info['qpu_access_time'] / 1000
#print('QPU time used (ms): {:.1f}\n'.format(hyb_time))
#print(sampleset4) 

In [18]:
#for r in result_info(sampleset4,s,t):
#    print('Route '+r[0]+': '+r[1]+', weight '+str(r[2]))

### Timings

In [19]:
print('Construting QUBO: {:.3f}'.format(qubo_time))
print('Construting BQM: {:.3f}'.format(bqm_time))
print('\nLocal deterministic solver: {:.1f}'.format(det_time))
print('Local heuristic solver: {:.1f}'.format(heur_time))
print('Quantum solver: {:.1f}'.format(qpu_time))

Construting QUBO: 0.235
Construting BQM: 0.631

Local deterministic solver: 1.7
Local heuristic solver: 90.1
Quantum solver: 202.1


## More complex graph

Parameters with several equal lengths: vertices=10 s=2 t=8

In [20]:
seed = 42
vertices2 = 10
random.seed(seed)
G2 = nx.gnp_random_graph(vertices2, 0.30, seed, directed=True)
nx.set_edge_attributes(G2, {e: {'weight': random.randint(1, 10)} for e in G2.edges})

s2 = 2
t2 = 8

In [21]:
E2 = [] 
for e in G2.edges(data=True):
    E2.append((e[0],e[1],e[2]['weight']))
print('Number of edges:',len(E2))
print('Number of vertices:',vertices2)

Number of edges: 34
Number of vertices: 10


### Test with classical algorithm

In [22]:
ts = time.time()
res = nx.all_shortest_paths(G2,s2,t2,weight='weight')
classical_time = (time.time()-ts)*1000
print('Time used by classical algorithm (ms): {:.3f}\n'.format(classical_time))
for r in res:
    print(r)

Time used by classical algorithm (ms): 0.160

[2, 9, 8]
[2, 9, 7, 8]


### QUBO and BQM

In [23]:
max_count2 = 0
labels2 = {}
for i,e in enumerate(E2):
    max_count2 += e[2]
    labels2[i] = str(e[0]) + '-' + str(e[1])
print('Max count:',max_count2)

Max count: 166


In [25]:
ts = time.time()
p = max_count2
Q2 = create_qubo(E2,p,s2,t2)
qubo_time = (time.time()-ts) * 1000
print('Time used for construction Q (ms): {:.3f}'.format(qubo_time))

ts = time.time()
bqm2 = dimod.BinaryQuadraticModel(Q2, 'BINARY')
bqm_time = (time.time()-ts) * 1000
bqm2 = bqm2.relabel_variables(labels2, inplace=False)
print('Time used for construction BQM (ms): {:.3f}'.format(bqm_time))

Time used for construction Q (ms): 0.677
Time used for construction BQM (ms): 0.304


### Local heuristic solver

In [26]:
num_reads=400
t1 = time.time()
sampleset5 = SimulatedAnnealingSampler().sample(bqm2, num_reads=num_reads).aggregate()
heur_time = (time.time()-t1) * 1000
print('Time used (ms): {:.3f}\n'.format(heur_time))

print('Lowest energy should be nearly:',-p)  
print('Lowest energy reached:',int(sampleset5.first.energy))
print('Lowest energy occurences: {:.1f} %'.format(sampleset5.first.num_occurrences/num_reads*100))

Time used (ms): 291.862

Lowest energy should be nearly: -166
Lowest energy reached: -161
Lowest energy occurences: 3.0 %


In [27]:
print(sampleset5.filter(lambda s: s.energy<0).truncate(10))

  0-2 0-3 0-4 0-8 1-0 1-2 1-4 1-5 1-8 2-1 2-6 2-9 3-0 ... 9-8 energy num_oc.
0   0   0   0   0   0   0   0   0   0   0   0   1   0 ...   0 -161.0      12
1   0   0   0   0   0   0   0   0   0   0   0   1   0 ...   1 -161.0      20
2   0   0   0   0   0   0   0   0   0   0   0   1   0 ...   0 -156.0      10
3   0   0   0   1   0   0   0   0   0   0   0   1   0 ...   0 -155.0      10
4   0   0   0   0   0   0   0   0   0   0   0   1   0 ...   0 -155.0      13
5   0   0   0   0   0   0   0   0   1   1   0   0   0 ...   0 -154.0      12
6   0   0   0   0   0   0   0   0   0   0   1   0   0 ...   0 -151.0      17
7   0   0   1   0   0   0   0   0   0   0   0   1   0 ...   0 -150.0       9
8   0   0   0   0   0   0   0   0   0   0   1   0   0 ...   0 -150.0      10
9   0   0   0   0   0   0   1   0   0   1   0   0   0 ...   0 -150.0      10
['BINARY', 10 rows, 123 samples, 34 variables]


In [28]:
for r in result_info(sampleset5,s2,t2,E2):
    print('Route '+r[0]+': '+str(r[1])+', weight '+str(r[2]))

Route 2-8: [2, 9, 7, 8], weight 5
Route 2-8: [2, 9, 8], weight 5


### Quantum solver

In [30]:
print('Number of logical qubits needed:',Q2.shape[0])
print('Number of logical couplers needed:', len(bqm2.quadratic))

Number of logical qubits needed: 34
Number of logical couplers needed: 202


In [31]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [32]:
num_reads = 5000
sampleset6 = EmbeddingComposite(machine).sample(bqm2, num_reads=num_reads)

In [33]:
qpu_time = sampleset6.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset6.info['embedding_context']['embedding'].values())
print('Lowest energy should be nearly:',-p)  
print('Lowest energy reached:',int(sampleset6.first.energy))
print('Lowest energy occurences: {:.1f} %'.format(sampleset6.first.num_occurrences/num_reads*100))
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}'.format(qubits))

Lowest energy should be nearly: -166
Lowest energy reached: -161
Lowest energy occurences: 0.0 %
QPU time used (ms): 591.1
Physical qubits used: 108


In [34]:
for r in result_info(sampleset6.aggregate(),s2,t2,E2):
    print('Route '+r[0]+': '+str(r[1])+', weight '+str(r[2]))

Route 2-8: [2, 9, 8], weight 5


### Timings

In [35]:
print('Construting QUBO (ms): {:.3f}'.format(qubo_time))
print('Construting BQM (ms): {:.3f}'.format(bqm_time))
print('\nLocal heuristic solver (ms): {:.1f}'.format(heur_time))
print('Quantum solver (ms): {:.1f}'.format(qpu_time))

Construting QUBO (ms): 0.677
Construting BQM (ms): 0.304

Local heuristic solver (ms): 291.9
Quantum solver (ms): 591.1


## Test procedure

In [36]:
def test_algorithm(E,solver,vertices,num_reads):
    max_count = 0
    labels = {}
    for i,e in enumerate(E):
        max_count += e[2]
        labels[i] = str(e[0]) + '-' + str(e[1])
    G = make_G(E,vertices)
    ok = True
    c = 0
    for s in range(vertices):
        for t in range(vertices):
            if s!=t and nx.has_path(G,s,t):
                c += 1
                Q = create_qubo(E,max_count,s,t)
                bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
                bqm = bqm.relabel_variables(labels, inplace=False)
                sampleset = None
                if num_reads==0:
                    sampleset = solver.sample(bqm).aggregate()
                else:
                    sampleset = solver.sample(bqm, num_reads=num_reads).aggregate()
                correct = [p for p in nx.all_shortest_paths(G,s,t,weight='weight')]
                result = result_info(sampleset,s,t,E)
                if len(correct)==len(result):
                    for r in result:
                        if r[1] not in correct:
                            ok = False
                else:
                    ok=False
    if ok:
        print('Test cases: {} (passed)'.format(c))
    else:
        print('Test cases: {} (failed)'.format(c))

print('Test with deterministic solver')
test_algorithm(E1, dimod.ExactSolver(), vertices1, 0)
print('\nTest with heuristic solver')
test_algorithm(E1, SimulatedAnnealingSampler(), vertices1, 400)
test_algorithm(E2, SimulatedAnnealingSampler(), vertices2, 400)

Test with deterministic solver
Test cases: 13 (passed)

Test with heuristic solver
Test cases: 13 (passed)
Test cases: 90 (passed)
